In [12]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.preprocessing import StandardScaler
from haversine import haversine

%matplotlib inline

In [8]:
rs = 42

In [9]:
df = pd.read_csv('transport_data.csv', na_values='-')
df.head()

,longitude,latitude,request_ts,trans_ts,label
0,30.295780,59.932167,1511956827,1511956594,NaN
1,30.219528,59.954617,1511956827,1511956746,NaN
2,30.409101,59.931679,1511956827,1511956747,NaN
3,30.293602,59.938892,1511956827,1511956759,?
4,30.360102,59.931103,1511956827,1511956767,0


In [13]:
scale = StandardScaler()
# X_nn = df.dropna().loc[df.label!='?'].drop(['request_ts', 'trans_ts', 'label'], axis=1)
X_nn = df.dropna().drop_duplicates(['longitude', 'latitude', 'trans_ts', 'label']).loc[df.label != '?'].loc[:, [
    'longitude', 'latitude',
]]
y_nn = df.dropna().drop_duplicates(['longitude', 'latitude', 'trans_ts', 'label']).loc[df.label != '?'].label.astype(int)
X_norm_nn = scale.fit_transform(X_nn)

In [10]:
kfold = StratifiedKFold(n_splits=5, random_state=rs)

In [15]:
nn = KNeighborsClassifier(n_neighbors=1, n_jobs=-1, metric=haversine)
cv = cross_val_score(nn, X_norm_nn, y=y_nn, scoring=make_scorer(accuracy_score), cv=kfold)
nn.fit(X_nn, y_nn)

KNeighborsClassifier(algorithm='auto', leaf_size=30,
           metric=<function haversine at 0x7fb2e6969ae8>,
           metric_params=None, n_jobs=-1, n_neighbors=1, p=2,
           weights='uniform')

In [16]:
np.mean(cv)

0.47064195905732714

In [17]:
test_base = df.dropna().loc[df.label == '?'].loc[:, [
    'longitude', 'latitude',
#     'req_dayofweek', 'req_hour', 'req_minute',
#     'trans_dayofweek', 'trans_hour', 'trans_minute',
]]
test_base.info()
X_nn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 3 to 179067
Data columns (total 2 columns):
longitude    12000 non-null float64
latitude     12000 non-null float64
dtypes: float64(2)
memory usage: 281.2 KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 30333 entries, 4 to 179077
Data columns (total 2 columns):
longitude    30333 non-null float64
latitude     30333 non-null float64
dtypes: float64(2)
memory usage: 710.9 KB


In [18]:
one_nn_pr = nn.predict(test_base)

In [19]:
one_nn_pr

array([1, 1, 1, ..., 1, 1, 0])

In [ ]:
np.savetxt('data/one_nn.txt', one_nn_pr, fmt='%1i')